## Sesión #5


Programa:

1. Leer bases de datos. 
2. Resumir información. 
3. Reestructurar bases de datos. 
4. Trabajar con datos en formato de texto. 
5. Introducción a expresiones regulares (regex). 


## Referencias 

- McKinney, W. (2017). Python for data analysis: Data wrangling with Pandas, NumPy, and IPython. Segunda edición. "O'Reilly Media, Inc.".
- Sitio "Empezando con pandas": https://pandas.pydata.org/docs/getting_started/

# 1. Leer distintos tipos de bases de datos

In [1]:
# Importar pandas 
import pandas as pd

pandas presenta una serie de funciones para leer datos tabulares como un objeto DataFrame: 

| Función | Descripción |
| :--- | :--- |
| read_csv | Cargar datos delimitados desde un archivo, URL u objeto similar a un archivo que usa coma como delimitador predeterminado | 
| read_table | Cargar datos delimitados desde un archivo, URL u objeto similar a un archivo; use tab ('\ t') como delimitador predeterminado | 
| read_excel | Leer datos tabulares de un archivo Excel XLS o XLSX | 
| read_html | Leer todas las tablas que se encuentran en el documento HTML proporcionado |
| read_json | Leer datos en formato JSON (JavaScript Object Notation) |
| read_sas | Leer un conjunto de datos almacenado en uno de los formatos personalizados del sistema SAS|
| read_sql | Leer los resultados de una consulta SQL (usando SQLAlchemy) como un DataFrame de pandas |
| read_stata | Leer un conjunto de datos en formato de archivo Stata |

## Ejemplo 1 - Base bien comportada

Podemos previsualizar un documento .csv con el siguiente comando: 

In [17]:
# Sólo se recomienda esto para bases pequeñas
#!cat Bases\ex1.csv # se supone que este funciona en Mac
!type Bases\ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


Dado que está delimitado por comas, podemos usar `read_csv` para leerlo en un DataFrame:

In [24]:
df = pd.read_csv('Bases/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


También podríamos haber usado `read_table` y especificado el delimitador:

In [22]:
pd.read_table('Bases/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [11]:
# También se pueden leer csv desde github
df = pd.read_csv("https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/ex1.csv")
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## Ejemplo 2 - Base sin nombres de columnas

In [29]:
# Previsualizamos la base
!type Bases\ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


Tenemos varias opciones para importar la base, por ejemplo: 

In [30]:
# Dejar que pandas asigne un nombre por default a cada columna
pd.read_csv('Bases/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [31]:
# Asignarle nombres a cada columna
pd.read_csv('Bases/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suponga que desea que la columna `message` sea el índice del DataFrame, esto se puede indicar con el argumento `index_col`:

In [38]:
pd.read_csv('Bases/ex2.csv', names=['a', 'b', 'c', 'd', 'message'], 
            index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


### Ejemplo 3 - Saltar filas con información 
Considere la siguiente base: 

In [42]:
!type Bases\ex3.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [44]:
# Podemos omitir la primera, tercera y cuarta filas de un archivo con skiprows:
pd.read_csv('Bases/ex3.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## Ejemplo 4 - Missing values
El manejo de los valores perdidos es una parte importante y frecuentemente utilizada en el proceso de análisis de archivos. Los datos faltantes generalmente no están presentes (cadena vacía) o están marcados con algún valor *centinela*. De forma predeterminada, pandas usa un conjunto de *centinelas* comunes, como NA y NULL. 

In [45]:
# Considere la siguiente base
!type Bases\ex4.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [46]:
# Veamos que pasa cuando la importamos
data = pd.read_csv('Bases/ex4.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [47]:
# Podemos ver qué valores toma como missing
pd.isnull(data)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


La opción `na_values` puede tomar una lista o un conjunto de cadenas para considerar los valores perdidos. Incluso se pueden especificar diferentes *centinelas* de NA para cada columna en un dict:

In [49]:
# Diccionario con centinelas
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
# Leer la base con nuestros missing
pd.read_csv('Bases/ex4.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


Las funciones para leer bases de datos tienen muchos argumentos adicionales para ayudar a manejar la amplia variedad de formatos de archivo excepcionales que llegan a surgir. 


Algunos argumentos de las funciones `read_csv`/`read_table`: 

| Argumento | Descripción |
| :--- | :--- |
| path | String que indica la localización del archivo en el sistema o URL | 
| sep o delimiter | Secuencia de caracteres o expresión regular para dividir entradas en cada fila |
| header | Número de fila para usar como nombres de columna; el valor predeterminado es 0 (primera fila), pero debería ser None si no hay una fila de encabezado | 
| index_col | Números de columna o nombres para usar como índice de fila en el resultado; puede ser un solo nombre / número o una lista de ellos para un índice jerárquico | 
| names | Lista de nombres de columna para el resultado, combinar con encabezado = Ninguno |
| skiprows | Número de filas al principio del archivo para ignorar o lista de números de fila para omitir. |
| na_values | Sequencia de valores para reemplazar con NA. | 
| nrows | Número de filas para leer desde el comienzo del archivo. |
| skip_footer | Número de líneas a ignorar al final del archivo. |
| encoding | Codificación de texto para Unicode (e.g., 'utf-8' para texto codificado en UTF-8). |
| thousands | Separador para miles (e.g., ',' or '.'). |

# Escribir datos en formato de texto
Los datos también se pueden exportar a un formato delimitado. Consideremos uno de los archivos CSV leídos antes:

In [52]:
data = pd.read_csv('Bases/ex4.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [53]:
# Usando el método to_csv de DataFrame, podemos escribir los datos en un archivo separado por comas:
data.to_csv('Bases/out.csv')

Por supuesto, se pueden usar otros delimitadores (escribimos `sys.stdout` para que imprima el resultado del texto en la consola en vez de crear otro archivo):

In [54]:
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


Los valores que faltan aparecen como valores en blanco en la salida. Es posible que desee denotarlos con algún otro valor, por ejemplo: 

In [56]:
data.to_csv(sys.stdout, sep='|', na_rep='NULL')

|something|a|b|c|d|message
0|one|1|2|3.0|4|NULL
1|two|5|6|NULL|8|world
2|three|9|10|11.0|12|foo


Sin otras opciones especificadas, se escriben las etiquetas de fila y columna por defecto, lo cual puede desactivarse:

In [57]:
data.to_csv(sys.stdout, sep='|', na_rep='NULL', 
            index=False, header=False)

one|1|2|3.0|4|NULL
two|5|6|NULL|8|world
three|9|10|11.0|12|foo


También puede escribir solo un subconjunto de las columnas y en el orden deseado:

In [59]:
data.to_csv(sys.stdout, sep='|', na_rep='NULL', 
            index=False, columns=['a', 'b', 'c'])

a|b|c
1|2|3.0
5|6|NULL
9|10|11.0


# Leer archivos de Microsoft Excel
pandas también admite la lectura de datos tabulares almacenados en archivos de Excel 2003 (y superior) utilizando la clase `ExcelFile` o la función `pandas.read_excel` (Internamente, estas herramientas utilizan los paquetes complementarios xlrd y openpyxl para leer archivos XLS y XLSX, respectivamente. Es posible que deba instalarlos manualmente con pip o conda).

In [67]:
data = pd.read_excel('Bases/ex5.xlsx', 'Sheet1')
data

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [71]:
help(pd.read_excel)

Help on function read_excel in module pandas.io.excel._base:

read_excel(io, sheet_name=0, header=0, names=None, index_col=None, usecols=None, squeeze=False, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skiprows=None, nrows=None, na_values=None, keep_default_na=True, verbose=False, parse_dates=False, date_parser=None, thousands=None, comment=None, skipfooter=0, convert_float=True, mangle_dupe_cols=True, **kwds)
    Read an Excel file into a pandas DataFrame.
    
    Supports `xls`, `xlsx`, `xlsm`, `xlsb`, and `odf` file extensions
    read from a local filesystem or URL. Supports an option to read
    a single sheet or a list of sheets.
    
    Parameters
    ----------
    io : str, bytes, ExcelFile, xlrd.Book, path object, or file-like object
        Any valid string path is acceptable. The string could be a URL. Valid
        URL schemes include http, ftp, s3, and file. For file URLs, a host is
        expected. A local file could be: ``file://loc

In [68]:
# También se pueden exportar las bases en formato de Excel
data.to_excel('Bases/out2.xlsx')

# JSON Data
JSON (abreviatura de JavaScript Object Notation) se ha convertido en uno de los formatos estándar para enviar datos mediante solicitud HTTP entre navegadores web y otras aplicaciones. Es un formato de datos de formato mucho más libre que un formato de texto tabular como CSV.


In [7]:
obj = """
{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
{"name": "Katie", "age": 38,
"pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

JSON es un código Python casi válido con la excepción de su valor nulo null y algunos otros matices (como no permitir las comas finales al final de las listas). Los tipos básicos son objetos (diccionarios), arrays (listas), strings, números, valores booleanos y nulos. Todas las claves de un objeto deben ser cadenas. Hay varias bibliotecas de Python para leer este tipo de archivos. Usaremos json aquí, ya que está integrado en la biblioteca estándar de Python. Para convertir una cadena JSON a formato Python, use json.loads:

In [9]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

La forma de convertir un objeto JSON o una lista de objetos en un DataFrame o alguna otra estructura de datos para el análisis dependerá de cada situación. Convenientemente, se puede pasar una lista de dicts (que anteriormente eran objetos JSON) al constructor de DataFrame y seleccionar un subconjunto de los campos de datos:

In [10]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


`pandas.read_json` puede convertir automáticamente conjuntos de datos JSON en arreglos específicos en una serie o DataFrame. Las opciones predeterminadas para `pandas.read_json` asumen que cada objeto en la matriz JSON es una fila en la tabla. 

# Web APIs
Muchos sitios web tienen API públicas que proporcionan fuentes de datos a través de JSON o algún otro formato. Hay varias formas de acceder a estas API desde Python, el método más utilizado es mediante el paquete `request`. 

## Ejemplo
Obtener datos de la serie histórica del indicador de Población total, en los Estados Unidos Mexicanos, en idioma español, en formato JSON y calcular su promedio.

**Se necesita un TOKEN personal.**

Link: https://www.inegi.org.mx/servicios/api_indicadores.html 

In [13]:
# Importamos las librerías necesarias
import requests
#import json

#Llamado al API
#url='https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000002/es/00000/false/BISE/2.0/[Aquí va tu Token]?type=json'
response= requests.get(url)
if response.status_code==200:
    content= json.loads(response.content)
    Series=content['Series'][0]['OBSERVATIONS']   
    
    #Obtención de la lista de observaciones 
    Observaciones=[]
    for obs in Series:  Observaciones.append(float(obs['OBS_VALUE']));
    

    #Generación del promedio de la lista de observaciones 
    sum=0.0
    for i in range(0,len(Observaciones)): sum=sum+Observaciones[i];  

    resultado=sum/len(Observaciones);
    print(resultado)

29893251.42857143


In [20]:
# Ver los objetos descargados
#content
Series

[{'TIME_PERIOD': '1910',
  'OBS_VALUE': '7504471.00000000000000000000',
  'OBS_EXCEPTION': None,
  'OBS_STATUS': '3',
  'OBS_SOURCE': '',
  'OBS_NOTE': '',
  'COBER_GEO': '0700'},
 {'TIME_PERIOD': '1921',
  'OBS_VALUE': '7003785.00000000000000000000',
  'OBS_EXCEPTION': None,
  'OBS_STATUS': '3',
  'OBS_SOURCE': '',
  'OBS_NOTE': '',
  'COBER_GEO': '0700'},
 {'TIME_PERIOD': '1930',
  'OBS_VALUE': '8119004.00000000000000000000',
  'OBS_EXCEPTION': None,
  'OBS_STATUS': '3',
  'OBS_SOURCE': '',
  'OBS_NOTE': '',
  'COBER_GEO': '0700'},
 {'TIME_PERIOD': '1940',
  'OBS_VALUE': '9695787.00000000000000000000',
  'OBS_EXCEPTION': None,
  'OBS_STATUS': '3',
  'OBS_SOURCE': '',
  'OBS_NOTE': '',
  'COBER_GEO': '0700'},
 {'TIME_PERIOD': '1950',
  'OBS_VALUE': '12696935.00000000000000000000',
  'OBS_EXCEPTION': None,
  'OBS_STATUS': '3',
  'OBS_SOURCE': '',
  'OBS_NOTE': '',
  'COBER_GEO': '0700'},
 {'TIME_PERIOD': '1960',
  'OBS_VALUE': '17415320.00000000000000000000',
  'OBS_EXCEPTION': None,
 

In [23]:
# Crear un dataframe con pandas
poblacion = pd.DataFrame(Series, columns=['TIME_PERIOD', 'OBS_VALUE'])
poblacion

,TIME_PERIOD,OBS_VALUE
0,1910,7504471.00000000000000000000
1,1921,7003785.00000000000000000000
2,1930,8119004.00000000000000000000
3,1940,9695787.00000000000000000000
4,1950,12696935.00000000000000000000
5,1960,17415320.00000000000000000000
6,1970,24065614.00000000000000000000
7,1980,33039307.00000000000000000000
8,1990,39893969.00000000000000000000
9,1995,44900499.00000000000000000000


In [30]:
# Calcular el promedio
pd.to_numeric(poblacion.OBS_VALUE, errors='coerce', downcast='integer').mean()

29893251.42857143

# 2. Resumir información

# Calcular estadísticas de una base de datos con pandas

Los objetos de pandas están equipados con un conjunto de métodos matemáticos y estadísticos comunes. La mayoría de estos se incluyen en la categoría de reducciones o estadísticas de resumen. Hay diversos métodos que extraen un solo valor (como la suma o la media) de una serie o una serie de valores de las filas o columnas de un DataFrame. En comparación con los métodos similares que se encuentran en las matrices NumPy, estos tienen un manejo integrado de los datos faltantes.

Por ahora usaremos como ejemplo la base de datos de pasajeros del Titanic, cuyas columnas contienen la siguiente información: 

- PassengerId: Id para cada pasajero.
- Survived: Esta característica tiene valor 0 y 1. 0 para no sobrevivido y 1 para sobrevivido.
- Pclass: Hay 3 clases: Clase 1, Clase 2 y Clase 3.
- Name: Nombre del pasajero.
- Sex: Género del pasajero.
- Age: Edad del pasajero.
- SibSp: Indicación de que el pasajero tiene hermanos y cónyuge.
- Parch: Si un pasajero está solo o tiene familia.
- Ticket: Número de ticket del pasajero.
- Fare: Tarifa pagada.
- Cabin: Cabina del pasajero. 
- Embarked: La categoría embarcada.

In [2]:
# Cargar y ver la base 
titanic = pd.read_csv("https://raw.githubusercontent.com/pandas-dev/pandas/master/doc/data/titanic.csv")
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Breve resumen del tipo de información en la base
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
# Revisamos columnas con datos nulos
nan_cols = titanic.isna().sum()
nan_cols[nan_cols>0]
#dict(titanic.isna().sum()) # forma alternativa

Age         177
Cabin       687
Embarked      2
dtype: int64

## Estadísticas "agregadas"
Estas son estadísticas que "agregan" los datos para obtener un valor específico, tales como la suma, la media, etc. 

In [ ]:
# ¿Cuál es el promedio de edad de los pasajeros del titanic? 
# titanic.Age.mean() # forma equivalente
titanic["Age"].mean()

Hay diferentes estadísticas disponibles y se pueden aplicar a columnas con datos numéricos. Las operaciones en general excluyen los datos faltantes y operan en filas de forma predeterminada.

In [9]:
# ¿Cuál es la edad promedio y el precio de la tarifa del boleto de los pasajeros del Titanic?
titanic[["Age", "Fare"]].median()
# El estadístico aplicado a varias columnas de un DataFrame se calcula para cada columna numérica.

Age     28.0000
Fare    14.4542
dtype: float64

El método `describe()` proporciona una descripción general rápida de los datos numéricos en un DataFrame. Cuando las columnas son datos en formato texto, el método `describe()` no los tiene en cuenta por defecto.

In [5]:
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


En lugar de las estadísticas predefinidas, se pueden definir combinaciones específicas de estadísticas de agregación para columnas determinadas mediante el método `.agg()`:

In [10]:
titanic.agg(
    {
        "Age": ["min", "max", "median", "skew"],
        "Fare": ["min", "max", "median", "mean"],
    }
)

,Age,Fare
max,80.000000,512.329200
mean,NaN,32.204208
median,28.000000,14.454200
min,0.420000,0.000000
skew,0.389108,NaN


## Estadísticas agregadas agrupadas por categoría
El cálculo de una estadística determinada (por ejemplo, la edad promedio) para cada categoría en una columna (por ejemplo, hombre/mujer en la columna Sex) es un patrón común. El método `groupby` se utiliza para realizar este tipo de operaciones. De manera más general, esto encaja en el patrón más general de dividir-aplicar-combinar (*split-apply-combine*):

- Divida los datos en grupos
- Aplicar una función a cada grupo de forma independiente
- Combinar los resultados en una estructura de datos

In [11]:
# ¿Cuál es la edad promedio de los pasajeros del Titanic por sexo?
titanic[["Sex", "Age"]].groupby("Sex").mean()
# Como nuestro interés es la edad promedio para cada género, 
# primero se hace una subselección en estas dos columnas: titanic[["Sex", "Age"]]. 
# Segundo, se aplica el método groupby() en la columna Sexo para crear un grupo por categoría. 
# Finalmente, se calcula y se devuelve la edad promedio de cada sexo.

,Age
Sex,
female,27.915709
male,30.726645


En el ejemplo anterior, primero seleccionamos explícitamente las 2 columnas. De lo contrario, el método `mean()` se aplica a cada columna que contiene columnas numéricas:

In [12]:
titanic.groupby("Sex").mean()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
Sex,,,,,,,
female,431.028662,0.742038,2.159236,27.915709,0.694268,0.649682,44.479818
male,454.147314,0.188908,2.389948,30.726645,0.429809,0.235702,25.523893


Si solo estamos interesados en la edad promedio para cada género, la selección de columnas (corchetes rectangulares [] como es habitual) también se admite en los datos agrupados:

In [13]:
titanic.groupby("Sex")["Age"].mean()

Sex
female    27.915709
male      30.726645
Name: Age, dtype: float64

La columna Pclass contiene datos numéricos pero en realidad representa 3 categorías (o factores) con las etiquetas "1", "2" y "3" respectivamente. Calcular estadísticas sobre estos no tiene mucho sentido. Por lo tanto, pandas proporciona un tipo de datos categórico para manejar este tipo de datos.

En ese sentido, podríamos preguntarnos ¿Cuál es el precio promedio de la tarifa del boleto para cada una de las combinaciones de clase de cabina y sexo?

In [16]:
titanic.groupby(["Sex", "Pclass"])["Fare"].mean()

Sex     Pclass
female  1         106.125798
        2          21.970121
        3          16.118810
male    1          67.226127
        2          19.741782
        3          12.661633
Name: Fare, dtype: float64

### Contar números por categoría 
El método `value_counts()` cuenta el número de registros para cada categoría en una columna. Tal función es un atajo, ya que en realidad es una operación de grupo en combinación con el recuento del número de registros dentro de cada grupo. 

In [18]:
# ¿Cuál es el número de pasajeros en cada una de las clases?
titanic["Pclass"].value_counts()
titanic.groupby("Pclass")["Pclass"].count()
# Ambos códigos proporcinan el mismo resultado

3    491
1    216
2    184
Name: Pclass, dtype: int64

Tanto `size` como `count` se pueden usar en combinación con `groupby`. Mientras que `size` incluye valores NaN y solo proporciona el número de filas (tamaño de la tabla), `count` excluye los valores faltantes. En el método `value_counts`, se puede usar el argumento `dropna` para incluir o excluir los valores de NaN.

# 3. Reestructurar bases de datos 
Hay una serie de operaciones básicas para reorganizar los datos tabulares. Estos se denominan alternativamente operaciones de *reshape* o *pivot*. 

## Reorganizar con índices jerárquicos
La indexación jerárquica proporciona una forma coherente de reorganizar los datos en un DataFrame. Hay dos acciones principales:

- *stack* o *apilar*: para "rota" o "pivota" de las columnas en los datos a las filas.
- *unstack* o *desapilar*: para "pivotar" de las filas a las columnas.

In [66]:
# Considere un df con índices de fila y columna:
t1 = titanic.groupby(["Pclass"])[["Fare", "Age"]].mean()
t1

,Fare,Age
Pclass,,
1,84.154687,38.233441
2,20.662183,29.877630
3,13.675550,25.140620


Utilizando el método `stack()` sobre este conjunto de datos "pivota" los datos en las columnas hacia las filas, produciendo un objeto `pd.series`:

In [68]:
t1_apilado = t1.stack()
t1_apilado

Pclass      
1       Fare    84.154687
        Age     38.233441
2       Fare    20.662183
        Age     29.877630
3       Fare    13.675550
        Age     25.140620
dtype: float64

Desde una serie con índices jerárquicos se pueden reorganizar los datos de nuevo en un `DataFrame` con `unstack()`:

In [71]:
t1_apilado.unstack()

,Fare,Age
Pclass,,
1,84.154687,38.233441
2,20.662183,29.877630
3,13.675550,25.140620


Por defecto, el nivel más bajo es "desapilado", pero esto puede modificarse fácilmente indicando el nombre de la columna que se desea desapilar: 

In [72]:
t1_apilado.unstack("Pclass")

Pclass,1,2,3
Fare,84.154687,20.662183,13.67555
Age,38.233441,29.877630,25.14062


## Reordenar DataFrames
Cuando se trata de objetos `pd.DataFrame` las funciones principales son `pivot` y .. 

### Pivotear del formato “Long” hacia “Wide”
Una forma común de almacenar múltiples series de tiempo en bases de datos y CSV es en el llamado formato largo o apilado. 

Un ejemplo de formato largo es el siguiente: 

In [74]:
# La siguiente tabla agrupa por sexo y por clase la tarifa promedio pagada por cada pasajero
t2 = titanic.groupby(["Sex", "Pclass"])["Fare"].mean().reset_index()
## Usamos el método reset_index() para que utilice las variables agrupadas como variables (columnas) en vez de índices
## Esto es necesario para usar pivot 
t2

,Sex,Pclass,Fare
0,female,1,106.125798
1,female,2,21.970121
2,female,3,16.118810
3,male,1,67.226127
4,male,2,19.741782
5,male,3,12.661633


Suponga que deseamos ver esta tabla en formato *wide* o ancho. El método *pivot* está diseñado para realizar esa tarea: 

In [75]:
t2.pivot(index="Sex", columns="Pclass", values="Fare")

Pclass,1,2,3
Sex,,,
female,106.125798,21.970121,16.118810
male,67.226127,19.741782,12.661633


Los dos primeros argumentos son las columnas que se usarán respectivamente como índice de fila y columna, luego finalmente una columna de valor opcional para llenar el DataFrame. Al omitir el último argumento, obtiene un DataFrame con columnas jerárquicas:

In [80]:
t2.pivot(index="Sex", columns="Pclass")

Fare                      
Pclass           1          2          3
Sex                                     
female  106.125798  21.970121  16.118810
male     67.226127  19.741782  12.661633

In [77]:
# Lo mismo aplica para tablas más complicadas, por ejemplo: 
t3 = titanic.groupby(["Sex", "Pclass"]).agg(
        {
        "Age": ["mean"],
        "Fare": ["mean"],
        "PassengerId": ["count"],
        "Survived": ["sum"]    
        }
).reset_index()
t3

,Sex,Pclass,Age,Fare,PassengerId,Survived
,,,mean,mean,count,sum
0,female,1,34.611765,106.125798,94,91
1,female,2,28.722973,21.970121,76,70
2,female,3,21.750000,16.118810,144,72
3,male,1,41.281386,67.226127,122,45
4,male,2,30.740707,19.741782,108,17
5,male,3,26.507589,12.661633,347,47


In [81]:
# En formato wide sería, por ejemplo:
t3.pivot("Pclass", "Sex", ["Age", "Fare", "PassengerId", "Survived"])

Age                   Fare            PassengerId         \
Sex        female       male      female       male      female   male   
Pclass                                                                   
1       34.611765  41.281386  106.125798  67.226127        94.0  122.0   
2       28.722973  30.740707   21.970121  19.741782        76.0  108.0   
3       21.750000  26.507589   16.118810  12.661633       144.0  347.0   

       Survived        
Sex      female  male  
Pclass                 
1          91.0  45.0  
2          70.0  17.0  
3          72.0  47.0

In [79]:
t3.pivot("Pclass", "Sex")

Age                   Fare            PassengerId      Survived  \
             mean                   mean                  count           sum   
Sex        female       male      female       male      female male   female   
Pclass                                                                          
1       34.611765  41.281386  106.125798  67.226127          94  122       91   
2       28.722973  30.740707   21.970121  19.741782          76  108       70   
3       21.750000  26.507589   16.118810  12.661633         144  347       72   

             
             
Sex    male  
Pclass       
1        45  
2        17  
3        47

## Pivotear del formato “Wide” hacia “Long”
Una operación inversa para `pivot` para `DataFrames` es `melt`. En lugar de transformar una columna en muchas en un nuevo `DataFrame`, fusiona varias columnas en una, produciendo un `DataFrame` que es más largo que la entrada.

In [102]:
# Considere el ejemplo que teníamos antes
twide = t2.pivot(index="Sex", columns="Pclass", values="Fare").reset_index()
# Nótese que agregamos reset_index() porque melt necesita que las variables sean columnas y no índices
twide

Pclass,Sex,1,2,3
0,female,106.125798,21.970121,16.118810
1,male,67.226127,19.741782,12.661633


In [108]:
twide.melt(id_vars=["Sex"])

,Sex,Pclass,value
0,female,1,106.125798
1,male,1,67.226127
2,female,2,21.970121
3,male,2,19.741782
4,female,3,16.118810
5,male,3,12.661633


In [112]:
twide2 = t3.pivot("Pclass", "Sex", ["Age", "Fare", "PassengerId", "Survived"]).reset_index()
twide2

Pclass        Age                   Fare            PassengerId         \
Sex            female       male      female       male      female   male   
0        1  34.611765  41.281386  106.125798  67.226127        94.0  122.0   
1        2  28.722973  30.740707   21.970121  19.741782        76.0  108.0   
2        3  21.750000  26.507589   16.118810  12.661633       144.0  347.0   

    Survived        
Sex   female  male  
0       91.0  45.0  
1       70.0  17.0  
2       72.0  47.0

In [114]:
twide2.melt(["Pclass"])

,Pclass,NaN,Sex,value
0,1,Age,female,34.611765
1,2,Age,female,28.722973
2,3,Age,female,21.750000
3,1,Age,male,41.281386
4,2,Age,male,30.740707
5,3,Age,male,26.507589
6,1,Fare,female,106.125798
7,2,Fare,female,21.970121
8,3,Fare,female,16.118810
9,1,Fare,male,67.226127


# Trabajar con datos en formato de texto 


## Convertir a minúsculas
Para convertir cada uno de los strings de una columna en minúsculas, se selecciona la columna deseada, se agrega el descriptor de acceso `str` y se aplica el método `lower`. 

Hay varios métodos para stings especializados disponibles cuando se usa el descriptor de acceso `str`. Estos métodos tienen, en general, nombres coincidentes con los métodos para strings incorporados en python equivalentes para elementos individuales, pero se aplican elemnt-wise en cada uno de los valores de las columnas.

In [32]:
# Convertir en minúsculas los nombres de las personas en la base 
titanic["Name"].str.lower()

0                                braund, mr. owen harris
1      cumings, mrs. john bradley (florence briggs th...
2                                 heikkinen, miss. laina
3           futrelle, mrs. jacques heath (lily may peel)
4                               allen, mr. william henry
                             ...                        
886                                montvila, rev. juozas
887                         graham, miss. margaret edith
888             johnston, miss. catherine helen "carrie"
889                                behr, mr. karl howell
890                                  dooley, mr. patrick
Name: Name, Length: 891, dtype: object

In [43]:
titanic["Name"].str.contains("Jack")

0      False
1      False
2      False
3      False
4      False
       ...  
886    False
887    False
888    False
889    False
890    False
Name: Name, Length: 891, dtype: bool

## Dividir srtings
También se pueden dividir strings. 

Ejemplo: crear una nueva columna `Surname` que contenga el apellido de los pasajeros extrayendo la parte antes de la coma.

In [51]:
# Se pueden dividir los string el método split
titanic["Name"].str.split(",")
# Nótese que como resultado los valores se devuelve en una lista con 2 elementos. 
# El primer elemento es la parte anterior a la coma y el segundo elemento es la parte posterior a la coma.

0                             [Braund,  Mr. Owen Harris]
1      [Cumings,  Mrs. John Bradley (Florence Briggs ...
2                              [Heikkinen,  Miss. Laina]
3        [Futrelle,  Mrs. Jacques Heath (Lily May Peel)]
4                            [Allen,  Mr. William Henry]
                             ...                        
886                             [Montvila,  Rev. Juozas]
887                      [Graham,  Miss. Margaret Edith]
888          [Johnston,  Miss. Catherine Helen "Carrie"]
889                             [Behr,  Mr. Karl Howell]
890                               [Dooley,  Mr. Patrick]
Name: Name, Length: 891, dtype: object

Como solo nos interesa la primera parte que representa el apellido (elemento 0), podemos usar nuevamente el descriptor de acceso `str` y aplicar `Series.str.get()` para extraer la parte relevante. De hecho, estas funciones de cadena se pueden concatenar para combinar múltiples funciones a la vez.

In [48]:
titanic["Surname"] = titanic["Name"].str.split(",").str.get(0)
titanic["Surname"]

0         Braund
1        Cumings
2      Heikkinen
3       Futrelle
4          Allen
         ...    
886     Montvila
887       Graham
888     Johnston
889         Behr
890       Dooley
Name: Surname, Length: 891, dtype: object

## Contención de valores 
El método `Series.str.contains()` comprueba cada uno de los valores de la columna indicada si contiene la palabra deseada y devuelve para cada uno de los valores Verdadero o Falso. Esta salida se puede utilizar para subseleccionar los datos mediante la indexación condicional (booleana). 


In [58]:
# ¿Viajaba algún Jack en el titanic? 
#titanic["Name"].str.contains("Jack")
titanic[titanic["Name"].str.contains("Jack")]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname
766,767,0,1,"Brewe, Dr. Arthur Jackson",male,NaN,0,0,112379,39.6,NaN,C,Brewe


In [62]:
# ¿Viajaba alguna Rose en el titanic? 
titanic[titanic["Name"].str.contains("Rose")]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname
855,856,1,3,"Aks, Mrs. Sam (Leah Rosen)",female,18.0,0,1,392091,9.35,NaN,S,Aks


## Reemplazar valores
El método `replace()` proporciona una forma conveniente de usar mapeos o vocabularios para traducir ciertos valores. Requiere un diccionario para definir el mapeo `{from: to}`.

In [63]:
# En la columna "Sex", reemplazamos los valores de "masculino" por "M" y los valores de "femenino" por "F"
titanic["Sex_short"] = titanic["Sex"].replace({"male": "M", "female": "F"})
titanic["Sex_short"]

0      M
1      F
2      F
3      F
4      M
      ..
886    M
887    F
888    F
889    M
890    M
Name: Sex_short, Length: 891, dtype: object

También hay un método `replace()` disponible para reemplazar un conjunto específico de caracteres. Sin embargo, al tener un mapeo de múltiples valores, esto se complica y es fácil equivocarse, por lo que sólo se recomienda para modificar ciertos valores en específico. 

In [ ]:
# Análogo al ejemplo anterior con str.replace
titanic["Sex_short"] = titanic["Sex"].str.replace("female", "F")
titanic["Sex_short"] = titanic["Sex_short"].str.replace("male", "M")

# Introducción a expresiones regulares (regex)
Las expresiones regulares proporcionan una forma flexible de buscar o hacer coincidir patrones de strings (a menudo más complejos) en el texto. Una sola expresión, comúnmente llamada **regex**, es un string formado según el lenguaje de expresiones regulares. El módulo incorporado en Python es responsable de aplicar expresiones regulares a los strings, pero también pandas incluye diversas funciones obtimizadas para bases de datos. 

Las funciones del módulo `re` se dividen en tres categorías: coincidencia de patrones, sustitución y división. Naturalmente, todos estos están relacionados; una expresión regular describe un patrón para ubicar en el texto, que luego se puede usar para muchos propósitos. 

Ejemplo: supongamos que queremos dividir una cadena de texto con un número variable de caracteres de espacio en blanco (tabulaciones, espacios y nuevas líneas). La expresión regular que describe uno o más espacios en blanco es \s+:

Algunas expresiones regulares 

| Símbolo | Significado |
| :--- | :--- |
| ^ | Inicia con cierto string |
| $ | Termina con cierto string |
| \s | Espacio en blanco |

https://cheatography.com/davechild/cheat-sheets/regular-expressions/pdf/

In [37]:
import re
text = "foo bar\t baz \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

Cuando utiliza `re.split('\s+', text)`, la expresión regular pimero se compila y luego se llama a su método split en el texto pasado. Si, en cambio, queremos obtener una lista de todos los patrones que coinciden con la expresión regular, se puede usar la función `findall`: 

In [36]:
re.findall('\s+', text)

# o en forma equivalente
#regex = re.compile('\s+')
#regex.findall(text)

[' ', '\t ', ' \t']

`match` y `search` están estrechamente relacionadas con `findall`. Mientras que `findall` devuelve todas las coincidencias en una cadena, `search` devuelve solo la primera coincidencia. De manera más rígida, `mathc` hace coincidir solo al principio de la cadena. 

Considere el siguiente ejemplo: 

In [15]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE para ingnorar mayúsculas y minúsculas
regex = re.compile(pattern, flags=re.IGNORECASE)

In [17]:
# El uso de findall en el texto produce una lista de las direcciones de correo electrónico:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [19]:
# Search devuelve la primera dirección de correo electrónico en el texto
regex.search(text)

<re.Match object; span=(5, 20), match='dave@google.com'>

In [24]:
# regex.match devuelve None, ya que solo coincidirá si el patrón ocurre al comienzo de la cadena
print(regex.match(text))

None


In [23]:
# sub devolverá un nuevo string reemplazado las coincidencias por un nuevo string:
print(regex.sub('email', text))

Dave email
Steve email
Rob email
Ryan email



## Funciones para string vectorizadas en pandas
Como ya vimos, las funciones o métodos especializados para strings en pandas están contenidas en el módulo `str`. Muchas de estas funciones son análogas a las que vienen incorporadas en python base. 

In [26]:
# Ejemplo en formato Series 
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com', 'Rob': 'rob@gmail.com', 'Wes': 'wes@hotmail.com'}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes      wes@hotmail.com
dtype: object

In [28]:
# Con el mismo patrón de antes, podemos extraer todos los correos de la base 
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [dave@google.com]
Steve    [steve@gmail.com]
Rob        [rob@gmail.com]
Wes      [wes@hotmail.com]
dtype: object